In [ ]:
%pip install scikit-learn imbalanced-learn==0.11.0
%pip install matplotlib
%pip install seaborn
%pip install mlflow dagshub
%pip install xgboost

In [7]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import warnings
import dagshub
import mlflow
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ieee-fraud-detection/sample_submission.csv
/kaggle/input/ieee-fraud-detection/test_identity.csv
/kaggle/input/ieee-fraud-detection/train_identity.csv
/kaggle/input/ieee-fraud-detection/test_transaction.csv
/kaggle/input/ieee-fraud-detection/train_transaction.csv


In [8]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 15)
pd.set_option('display.width', None)
pd.set_option('display.expand_frame_repr', False)

# to ignore warnings in cells
warnings.filterwarnings('ignore')

# Authorize on DagsHub

In [12]:
dagshub.init(repo_owner='b3tameche', repo_name='kaggle-fraud-detection', mlflow=True)

Initialized MLflow to track repo "b3tameche/kaggle-fraud-detection"

Repository b3tameche/kaggle-fraud-detection initialized!

# Loading the best model

In [13]:
best_model_name = "best_model_xgboost"
best_model_version = "1"

best_model_uri = f'models:/{best_model_name}/{best_model_version}'

best_model = mlflow.sklearn.load_model(best_model_uri)

# Minor issue 

In [39]:
df_train_transaction = pd.read_csv("/kaggle/input/ieee-fraud-detection/train_transaction.csv")
df_train_identity = pd.read_csv("/kaggle/input/ieee-fraud-detection/train_identity.csv")
df_train = pd.merge(df_train_transaction, df_train_identity, on="TransactionID", how="left")
df_train.drop('TransactionID', axis=1, inplace=True)

In [23]:
df_test_transaction = pd.read_csv("/kaggle/input/ieee-fraud-detection/test_transaction.csv")
df_test_identity = pd.read_csv("/kaggle/input/ieee-fraud-detection/test_identity.csv")
df_test = pd.merge(df_test_transaction, df_test_identity, on="TransactionID", how="left")
df_test.drop('TransactionID', axis=1, inplace=True)

In [44]:
set(df_train_transaction.columns) - set(df_test_transaction.columns)

{'isFraud'}

In [45]:
set(df_train_identity.columns) - set(df_test_identity.columns)

{'id_01',
 'id_02',
 'id_03',
 'id_04',
 'id_05',
 'id_06',
 'id_07',
 'id_08',
 'id_09',
 'id_10',
 'id_11',
 'id_12',
 'id_13',
 'id_14',
 'id_15',
 'id_16',
 'id_17',
 'id_18',
 'id_19',
 'id_20',
 'id_21',
 'id_22',
 'id_23',
 'id_24',
 'id_25',
 'id_26',
 'id_27',
 'id_28',
 'id_29',
 'id_30',
 'id_31',
 'id_32',
 'id_33',
 'id_34',
 'id_35',
 'id_36',
 'id_37',
 'id_38'}

**In testset, id column names contain '-' instead of '_', I'll modify it here (could've done into pipeline too).**

# Submission

In [48]:
old_column_names = list(df_test.columns)
new_column_names = [col.replace('-', '_') for col in old_column_names]

In [50]:
df_test.columns = new_column_names

In [52]:
set(df_train.columns) - set(df_test.columns)

{'isFraud'}

In [53]:
y_test_pred = best_model.predict(df_test)

final = pd.DataFrame({
  'TransactionID': np.arange(3663549, 3663549 + len(y_test_pred)),
  'isFraud': y_test_pred
})

final.to_csv('submission.csv', index=False)

print(final.head())

   TransactionID  isFraud
0        3663549        0
1        3663550        0
2        3663551        0
3        3663552        0
4        3663553        0
